In [1]:
# Bone Fracture Classification on Kaggle

# 1. تثبيت المكتبات (إذا لم تكن مثبتة)
!pip install tensorflow matplotlib

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import matplotlib.pyplot as plt
import os
import numpy as np

# 2. تحميل البيانات (تأكد أنك أضفت dataset من Kaggle في قسم Data للـ Notebook)
data_dir = '/kaggle/input/bone-fracture-dataset/Bone fracture dataset/Bone fracture dataset/Dataset'

# 3. تجهيز البيانات
img_size = (224, 224)
batch_size = 32

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_data = datagen.flow_from_directory(
    data_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# 4. بناء نموذج CNN بسيط
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# 5. تدريب النموذج
history = model.fit(train_data, epochs=5, validation_data=val_data)

# 6. رسم دقة التدريب والتحقق
plt.plot(history.history['accuracy'], label='train accuracy')
plt.plot(history.history['val_accuracy'], label='val accuracy')
plt.legend()
plt.show()

# 7. حفظ النموذج
model.save('bone_fracture_model.h5')

# 8. التنبؤ على صورة جديدة (ارفع الصورة في Kaggle وغيّر path)
from tensorflow.keras.preprocessing import image

def predict_image(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)
    if prediction[0][0] > 0.5:
        print("📛 العظم مكسور")
    else:
        print("✅ العظم سليم")

# مثال على التنبؤ (غيّر المسار حسب الصورة التي رفعتها)
predict_image('/kaggle/input/bone-fracture-dataset/Bone fracture dataset/Bone fracture dataset/Dataset/normal/0.png')


2025-09-18 11:02:16.580840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758193336.888666      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758193336.972295      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/bone-fracture-dataset/Bone fracture dataset/Bone fracture dataset/Dataset'

In [ ]:
import os
print(os.listdir('/kaggle/input'))

In [ ]:

from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image

# تحميل النموذج المدرب
model = tf.keras.models.load_model('bone_fracture_model.h5')
img_size = (224, 224)

# دالة التنبؤ
def predict_image(img_path):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    return prediction

# دالة عند اختيار الصورة
def open_image():
    file_path = filedialog.askopenfilename()
    if file_path:
        # عرض الصورة في الواجهة
        img = Image.open(file_path)
        img = img.resize((250, 250))
        photo = ImageTk.PhotoImage(img)
        image_label.config(image=photo)
        image_label.image = photo

        # التنبؤ
        result = predict_image(file_path)
        if result > 0.5:
            result_label.config(text=f"📛 العظم مكسور (نسبة: {result:.2f})", fg="red")
        else:
            result_label.config(text=f"✅ العظم سليم (نسبة: {1 - result:.2f})", fg="green")

# إنشاء نافذة
window = tk.Tk()
window.title("تصنيف صور العظام")
window.geometry("300x400")

# عناصر الواجهة
btn = tk.Button(window, text="اختر صورة", command=open_image)
btn.pack(pady=10)

image_label = tk.Label(window)
image_label.pack()

result_label = tk.Label(window, text="", font=("Arial", 14))
result_label.pack(pady=10)

# تشغيل التطبيق
window.mainloop()